Image recognition is an exceedingly difficult problem
Found that a kind of model called convolution neural network can achieve reasonable performance
on hard visual recognition tasks - matching or exceeding human performance
Inception-v3 is trained for ImageNet using the 2012 data.
There are 1000 classes.
Tutorial will teach to use Inception-v3

Transer learning can be very useful in many applications

TF layers module provides high level API that makes it easy to construct a neural network.

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
# Convolutional Layer #1: Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function
# Pooling Layer #1: Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)
# Convolutional Layer #2: Applies 64 5x5 filters, with ReLU activation function
# Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2
# Dense Layer #1: 1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training)
# Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).

In [12]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
#     Input layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])
#     Convolution layer 1
    conv1 = tf.layers.conv2d(
            inputs = input_layer,
            filters=32,
            kernel_size=[5,5],
            padding='SAME',
            activation=tf.nn.relu)
#     Pooling layer 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
#     Convolution layer 2 and Pooling layer 2
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5,5],
            padding='SAME',
            activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
#     Dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)
#     Logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    loss = None
    train_op = None
#     Calculate Loss for both train and eval modes
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
#     Configure Train op
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")
#      Generate predictions
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits, name="softmax_tensor")
    }
#     Return a model fn obj
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [ ]:
# Our application logic will be here
if __name__ == '__main__':
    tf.app.run()